**Data loading and NLP preprocessing**

In [7]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import os

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab') # Download punkt_tab resource

def load_data(file_path):
    df = pd.read_csv("/content/cyberbullying_tweets.csv")
    # Binary label: 0 for 'Not_cyberbullying', 1 for all other types
    df['label'] = df['cyberbullying_type'].apply(lambda x: 0 if x == 'not_cyberbullying' else 1)
    return df[['tweet_text', 'label']]

def preprocess_text(text):
    # Lowercase, remove URLs/mentions/hashtags, punctuation
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower().strip()
    # Tokenize, remove stopwords, lemmatize
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)

if __name__ == "__main__":
    df = load_data('/content/cyberbullying_tweets.csv')
    df['processed_text'] = df['tweet_text'].apply(preprocess_text)
    # Create the directory if it doesn't exist
    output_dir = 'data'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    df.to_csv(os.path.join(output_dir, 'processed_data.csv'), index=False)
    print(f"Processed {len(df)} samples.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Processed 47692 samples.


**Train SVM, Random Forest, and BERT**

In [17]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import numpy as np

class CyberbullyingDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts.reset_index(drop=True)  # Reset index to ensure sequential access
        self.labels = labels.reset_index(drop=True)  # Reset index for labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx]) if pd.notna(self.texts[idx]) else ""  # Handle NaN
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text, add_special_tokens=True, max_length=self.max_len,
            return_token_type_ids=False, padding='max_length', truncation=True,
            return_attention_mask=True, return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def train_classical_models(X_train, X_test, y_train, y_test):
    # Fill NaN values with empty strings
    X_train = X_train.fillna('')
    X_test = X_test.fillna('')

    vectorizer = TfidfVectorizer(max_features=5000)
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    # SVM
    svm = SVC(kernel='linear')
    svm.fit(X_train_vec, y_train)
    svm_pred = svm.predict(X_test_vec)
    print("SVM Accuracy:", accuracy_score(y_test, svm_pred))
    print(classification_report(y_test, svm_pred))

    # Random Forest
    rf = RandomForestClassifier(n_estimators=100)
    rf.fit(X_train_vec, y_train)
    rf_pred = rf.predict(X_test_vec)
    print("RF Accuracy:", accuracy_score(y_test, rf_pred))
    print(classification_report(y_test, rf_pred))

    return svm, rf, vectorizer

def train_bert(df):
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

    # Reset indices after split to ensure sequential access
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        df['processed_text'], df['label'], test_size=0.2, random_state=42
    )
    train_texts = train_texts.reset_index(drop=True)
    val_texts = val_texts.reset_index(drop=True)
    train_labels = train_labels.reset_index(drop=True)
    val_labels = val_labels.reset_index(drop=True)

    train_dataset = CyberbullyingDataset(train_texts, train_labels, tokenizer)
    val_dataset = CyberbullyingDataset(val_texts, val_labels, tokenizer)

    training_args = TrainingArguments(
        output_dir='./bert_results',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        eval_strategy='epoch'  # Updated from evaluation_strategy
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    trainer.train()
    model.save_pretrained('./bert_model')
    tokenizer.save_pretrained('./bert_model')
    return model, tokenizer

if __name__ == "__main__":
    df = pd.read_csv('data/processed_data.csv')

    # Check label distribution
    print("Label distribution before split:")
    print(df['label'].value_counts())

    X_train, X_test, y_train, y_test = train_test_split(
        df['processed_text'], df['label'], test_size=0.2, random_state=42
    )

    print("\nLabel distribution in y_train:")
    print(y_train.value_counts())
    print("\nLabel distribution in y_test:")
    print(y_test.value_counts())

    print("Training Classical Models...")
    svm, rf, vectorizer = train_classical_models(X_train, X_test, y_train, y_test)

    print("\nTraining BERT...")
    bert_model, bert_tokenizer = train_bert(df)

    # Save classical models
    import joblib
    output_dir = 'models'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    joblib.dump(svm, os.path.join(output_dir, 'svm_model.pkl'))
    joblib.dump(rf, os.path.join(output_dir, 'rf_model.pkl'))
    joblib.dump(vectorizer, os.path.join(output_dir, 'tfidf_vectorizer.pkl'))

Label distribution before split:
label
1    39747
0     7945
Name: count, dtype: int64

Label distribution in y_train:
label
1    31832
0     6321
Name: count, dtype: int64

Label distribution in y_test:
label
1    7915
0    1624
Name: count, dtype: int64
Training Classical Models...
SVM Accuracy: 0.8607820526260614
              precision    recall  f1-score   support

           0       0.71      0.31      0.43      1624
           1       0.87      0.97      0.92      7915

    accuracy                           0.86      9539
   macro avg       0.79      0.64      0.68      9539
weighted avg       0.84      0.86      0.84      9539

RF Accuracy: 0.8429604780375302
              precision    recall  f1-score   support

           0       0.56      0.35      0.43      1624
           1       0.88      0.94      0.91      7915

    accuracy                           0.84      9539
   macro avg       0.72      0.65      0.67      9539
weighted avg       0.82      0.84      0.83      95

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.338300,0.366820
2,0.440600,0.448146
3,0.320000,0.330840


**Simple inference script**

In [19]:
import joblib
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
import re # Import re for preprocess_text function
from nltk.corpus import stopwords # Import stopwords for preprocess_text function
from nltk.tokenize import word_tokenize # Import word_tokenize for preprocess_text function
from nltk.stem import WordNetLemmatizer # Import WordNetLemmatizer for preprocess_text function
import nltk # Import nltk for preprocess_text function

# Assuming preprocess_text is available from the previous cell
def preprocess_text(text):
    # Lowercase, remove URLs/mentions/hashtags, punctuation
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower().strip()
    # Tokenize, remove stopwords, lemmatize
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)

def predict_classical(text, model_type='svm'):
    # Check if models are loaded
    if 'vectorizer' not in globals() or ('svm' not in globals() and 'rf' not in globals()):
        print("Classical models not loaded. Please run the training cell first.")
        return None

    processed_text = preprocess_text(text)
    vec = vectorizer.transform([processed_text])
    if model_type == 'svm':
        if 'svm' in globals():
            pred = svm.predict(vec)[0]
            return 'Bullying' if pred == 1 else 'Non-Bullying'
        else:
            print("SVM model not loaded.")
            return None
    else:
        if 'rf' in globals():
            pred = rf.predict(vec)[0]
            return 'Bullying' if pred == 1 else 'Non-Bullying'
        else:
            print("Random Forest model not loaded.")
            return None


def predict_bert(text):
    # Check if BERT model is loaded
    if 'bert_tokenizer' not in globals() or 'bert_model' not in globals():
        print("BERT model not loaded. Please run the training cell first.")
        return None

    inputs = bert_tokenizer.encode_plus(text, return_tensors='pt', max_length=128, truncation=True)
    outputs = bert_model(**inputs)
    pred = torch.argmax(outputs.logits).item()
    return 'Bullying' if pred == 1 else 'Non-Bullying'

if __name__ == "__main__":
    # Create the directory if it doesn't exist
    output_dir = 'models'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Check if model files exist before loading
    svm_model_path = os.path.join(output_dir, 'svm_model.pkl')
    rf_model_path = os.path.join(output_dir, 'rf_model.pkl')
    vectorizer_path = os.path.join(output_dir, 'tfidf_vectorizer.pkl')
    bert_model_path = './bert_model'

    if os.path.exists(svm_model_path) and os.path.exists(rf_model_path) and os.path.exists(vectorizer_path) and os.path.exists(bert_model_path):
        # Load classical models
        svm = joblib.load(svm_model_path)
        rf = joblib.load(rf_model_path)
        vectorizer = joblib.load(vectorizer_path)

        # Load BERT
        bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_path)
        bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_path)

        sample_tweet = "You are so ugly, go kill yourself!"
        print("SVM:", predict_classical(sample_tweet, 'svm'))
        print("RF:", predict_classical(sample_tweet, 'rf'))
        print("BERT:", predict_bert(sample_tweet))
    else:
        print("Model files not found. Please run the training cell (UhYATZPFJ4fh) first to train and save the models.")

SVM: Bullying
RF: Non-Bullying
BERT: Bullying
